<a href="https://colab.research.google.com/github/BoWarburton/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

### Replicate the lesson code.

In [0]:
# Import libraries
import pandas as pd

In [0]:
# Load dataset
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')

In [24]:
df.shape

(1460, 81)

In [0]:
pd.set_option('display.max_rows', 100)

In [26]:
df.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


In [27]:
df['BedroomAbvGr']

0       3
1       3
2       3
3       3
4       4
       ..
1455    3
1456    3
1457    4
1458    2
1459    3
Name: BedroomAbvGr, Length: 1460, dtype: int64

In [29]:
df['LotFrontage'].head(10)

0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
5    85.0
6    75.0
7     NaN
8    51.0
9    50.0
Name: LotFrontage, dtype: float64

In [11]:
df['MSZoning']

0       RL
1       RL
2       RL
3       RL
4       RL
        ..
1455    RL
1456    RL
1457    RL
1458    RL
1459    RL
Name: MSZoning, Length: 1460, dtype: object

In [12]:
pd.set_option('display.max_rows', 150)
df['LotFrontage'].value_counts(dropna=False)

NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
65.0      44
85.0      40
78.0      25
21.0      23
90.0      23
68.0      19
24.0      19
64.0      19
73.0      18
72.0      17
79.0      17
63.0      17
55.0      17
100.0     16
51.0      15
66.0      15
74.0      15
52.0      14
59.0      13
71.0      12
67.0      12
57.0      12
82.0      12
43.0      12
40.0      12
76.0      11
69.0      11
53.0      10
92.0      10
88.0      10
34.0      10
86.0      10
77.0       9
35.0       9
44.0       9
84.0       9
62.0       9
93.0       8
61.0       8
98.0       8
96.0       8
107.0      7
95.0       7
120.0      7
58.0       7
41.0       6
105.0      6
54.0       6
94.0       6
48.0       6
110.0      6
30.0       6
89.0       6
81.0       6
91.0       6
36.0       6
47.0       5
83.0       5
37.0       5
87.0       5
32.0       5
56.0       5
102.0      4
42.0       4
49.0       4
99.0       3
45.0       3
104.0      3
130.0      3
103.0      3
108.0      3

In [13]:
df.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [0]:
import numpy as np

In [31]:
# What type of data is NaN
type(np.NaN)

float

In [32]:
df_small = df[['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'SalePrice']].copy()
df_small.head()

,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice
0,856,856,854,208500
1,1262,1262,0,181500
2,920,920,866,223500
3,756,961,756,140000
4,1145,1145,1053,250000


In [0]:
# New column TotalSquareFootage by summing all the other SF
df_small['TotalSF'] = df_small['TotalBsmtSF'] + df_small['1stFlrSF'] + df_small['2ndFlrSF']

In [38]:
df_small.head()

,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice,TotalSF,PricePerSqFt
0,856,856,854,208500,2566,81.254871
1,1262,1262,0,181500,2524,71.909667
2,920,920,866,223500,2706,82.594235
3,756,961,756,140000,2473,56.611403
4,1145,1145,1053,250000,3343,74.783129


In [39]:
# New column 'PricePerSqFt' by dividing the price by the square footage
df_small['PricePerSqFt'] = df_small['SalePrice'] / df_small['TotalSF']
df_small.head()

,TotalBsmtSF,1stFlrSF,2ndFlrSF,SalePrice,TotalSF,PricePerSqFt
0,856,856,854,208500,2566,81.254871
1,1262,1262,0,181500,2524,71.909667
2,920,920,866,223500,2706,82.594235
3,756,961,756,140000,2473,56.611403
4,1145,1145,1053,250000,3343,74.783129


In [94]:
# Import LendingClub dataset
import pandas as pd
df = pd.read_csv('LoanStats_2017Q1.csv', skiprows=1, skipfooter=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [95]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,NaN,NaN,3600,3600,3600,36 months,7.49%,111.97,A,A4,Code/Compliance Inspector,10+ years,MORTGAGE,120000.0,Not Verified,Mar-2017,Issued,n,NaN,NaN,other,Other,467xx,IN,18.90,0,Aug-1992,1,NaN,96.0,18,1,5658,14.9%,33,w,3600.00,3600.00,0.00,0.00,...,3,2,0,9.0,NaN,2.0,NaN,0,1,4,2,3,17,12,16,4,18,0.0,0,0,6,100.0,0.0,1,0,93599,36506,9000,55699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15000,15000,15000,60 months,14.99%,356.78,C,C4,Senior Superintendent,10+ years,MORTGAGE,125000.0,Not Verified,Mar-2017,Issued,n,NaN,NaN,other,Other,064xx,CT,17.25,0,Sep-1989,0,NaN,NaN,11,0,53167,75.3%,18,w,15000.00,15000.00,0.00,0.00,...,16,16,2,16.0,NaN,16.0,NaN,0,5,7,6,8,6,8,10,7,11,0.0,0,0,0,100.0,66.7,0,0,148572,69364,60300,30224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,8400,8400,8400,36 months,11.39%,276.56,B,B3,IT Tech,8 years,MORTGAGE,50000.0,Source Verified,Mar-2017,Issued,n,NaN,NaN,other,Other,454xx,OH,15.63,0,Apr-2005,0,NaN,NaN,14,0,12831,30.3%,30,w,8400.00,8400.00,0.00,0.00,...,4,4,4,4.0,NaN,10.0,NaN,0,4,5,7,11,9,11,16,5,14,0.0,0,0,5,100.0,14.3,0,0,205671,51591,36100,36876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000,36 months,10.49%,130.00,B,B2,Dental hygienist,2 years,RENT,50000.0,Source Verified,Mar-2017,Issued,n,NaN,NaN,major_purchase,Major purchase,768xx,TX,33.61,1,Apr-2003,0,6.0,37.0,10,1,4388,33.2%,33,w,4000.00,4000.00,0.00,0.00,...,15,7,6,120.0,NaN,9.0,61.0,1,1,4,1,3,15,6,12,4,10,0.0,0,0,1,87.5,100.0,0,0,152852,136208,2500,139652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,6000,6000,6000,36 months,7.24%,185.93,A,A3,Program Manager,3 years,RENT,125000.0,Source Verified,Mar-2017,Current,n,NaN,NaN,credit_card,Credit card refinancing,981xx,WA,9.25,0,Jul-2004,1,54.0,NaN,16,0,9571,41.3%,27,w,5850.27,5850.27,164.21,164.21,...,7,3,0,7.0,NaN,4.0,NaN,9,3,3,5,5,22,5,5,3,16,0.0,0,0,3,59.1,20.0,0,0,72865,60622,23200,49665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Sum null values and sort from least to most
df.isnull().sum()

id                                     96779
member_id                              96779
loan_amnt                                  0
funded_amnt                                0
funded_amnt_inv                            0
                                       ...  
sec_app_open_il_6m                     95754
sec_app_num_rev_accts                  95754
sec_app_chargeoff_within_12_mths       95754
sec_app_collections_12_mths_ex_med     95754
sec_app_mths_since_last_major_derog    96423
Length: 122, dtype: int64

### Convert the term column from string to integer.

In [64]:
# Clean up int rate column
# Looking at first few values, seems to be either 3 year or 5 year loan term
# expressed in months
df['term'].head()

0    36
1    60
2    36
3    36
4    36
Name: term, dtype: int64

In [65]:
# Getting more detail, there are leading spaces
# maybe because there might be longer terms later
df['term'][:30]

0     36
1     60
2     36
3     36
4     36
5     36
6     36
7     36
8     36
9     36
10    36
11    36
12    36
13    36
14    60
15    36
16    36
17    36
18    36
19    60
20    36
21    36
22    36
23    36
24    36
25    60
26    60
27    36
28    36
29    60
Name: term, dtype: int64

In [60]:
df['term'][1].strip().strip(' months')

'60'

In [61]:
# Still a string of course, but it works, now make a function and try it
def term_to_int(cell_contents):
  return int(cell_contents.strip().strip(' months'))
term_to_int(df['term'][8])

36

In [0]:
# Do it all with apply()
df['term'] = df['term'].apply(term_to_int)

In [63]:
df['term'].head()

0    36
1    60
2    36
3    36
4    36
Name: term, dtype: int64

### Make a column named loan_status_is_great

In [66]:
# New column contains integer 1 if loan_status is "Current" or "Fully Paid."
# Else it should contain the integer 0.
small_df = df.head(20)
small_df['loan_status'].head(5)


0     Issued
1     Issued
2     Issued
3     Issued
4    Current
Name: loan_status, dtype: object

In [0]:
# Function to return integer 1 if loan_status "Current" or "Fully Paid"
# Else 0
def set_loan_status_is_great_flag (cell_contents):
  if cell_contents == 'Current':
    return 1
  elif cell_contents == 'Fully Paid':
    return 1
  else:
    return 0


In [70]:
# Try it
set_loan_status_is_great_flag(small_df['loan_status'][4])

1

In [71]:
small_df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,annual_inc,url,desc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,...,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,loan_status_is_great
count,0.0,0.0,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,0.0,0.0,20.000000,20.000000,20.000000,8.000000,4.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.0,20.0,20.0,20.000000,20.0,4.000000,20.0,0.0,0.0,20.0,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,1.0,18.000000,3.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.0,20.0,20.0,20.000000,20.000000,20.000000,20.000000,20.0,20.000000,20.000000,20.000000,20.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.000000
mean,NaN,NaN,12415.000000,12415.000000,12415.000000,39.600000,383.113000,92000.000000,NaN,NaN,20.570500,0.150000,0.550000,36.250000,80.500000,13.300000,0.200000,16353.400000,28.050000,12392.484500,12392.484500,24.521000,24.521000,22.515500,2.005500,0.0,0.0,0.0,25.228500,0.0,56.500000,1.0,NaN,NaN,0.0,517.600000,165488.600000,1.000000,3.850000,0.950000,...,14.400000,7.900000,2.200000,22.800000,16.0,8.111111,32.666667,0.700000,4.100000,5.250000,6.000000,8.650000,11.150000,8.750000,14.200000,5.250000,13.300000,0.0,0.0,0.0,2.450000,95.235000,30.175000,0.150000,0.0,213022.750000,58417.350000,30330.000000,56026.85000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000
std,NaN,NaN,7637.323519,7637.323519,7637.323519,8.792341,230.729805,42332.020977,NaN,NaN,8.017648,0.366348,0.944513,24.040145,29.354159,3.785012,0.410391,13658.337987,10.947122,7645.152044,7645.152044,79.918478,79.918478,73.494881,6.432347,0.0,0.0,0.0,82.214533,0.0,19.672316,0.0,NaN,NaN,0.0,2129.982787,162846.918668,1.169795,2.476734,1.050063,...,13.789393,6.406411,2.504732,27.937148,NaN,6.763995,24.664414,2.154555,2.593514,2.712059,2.790963,5.584518,6.192313,3.640055,9.254302,2.712059,3.785012,0.0,0.0,0.0,2.762055,9.541779,35.987423,0.366348,0.0,171203.152647,35798.104987,16889.582588,34855.34911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307794
min,NaN,NaN,3600.000000,3600.000000,3600.000000,36.000000,111.970000,25000.000000,NaN,NaN,6.060000,0.000000,0.000000,6.000000,37.000000,6.000000,0.000000,1818.000000,16.000000,3600.000000,3600.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,30.000000,1.0,NaN,NaN,0.0,0.000000,10041.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,16.0,0.000000,16.000000,0.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000,1.000000,6.000000,0.0,0.0,0.0,0.000000,59.100000,0.000000,0.000000,0.0,33985.000000,10041.000000,2500.000000,18185.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,6000.000000,6000.000000,6000.000000,36.000000,199.497500,59000.000000,NaN,NaN,16.845000,0.00

In [72]:
# small_df['loan_status_is_great'] = small_df['loan_status'].apply(set_loan_status_is_great_flag)
small_df['loan_status_is_great'].head(5)

0    0
1    0
2    0
3    0
4    1
Name: loan_status_is_great, dtype: int64

### Make last_pymnt_d_month and last_pymnt_d_year columns.

In [96]:
# Make last_pymnt_d_month and last_pymnt_d_year columns
# From last_pymnt_d column
df['last_pymnt_d'].head()


0         NaN
1         NaN
2         NaN
3         NaN
4    Apr-2017
Name: last_pymnt_d, dtype: object

In [97]:
df['issue_d'].value_counts()

Mar-2017    37181
Jan-2017    31835
Feb-2017    27763
Name: issue_d, dtype: int64

In [0]:
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)

In [99]:
pd.set_option('display.max_rows', 200)
df.dtypes

id                                            float64
member_id                                     float64
loan_amnt                                       int64
funded_amnt                                     int64
funded_amnt_inv                                 int64
term                                           object
int_rate                                       object
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                datetime64[ns]
loan_status                                    object
pymnt_plan                                     object
url                         

In [102]:
df = pd.read_csv('LoanStats_2017Q1.csv', skiprows=1, skipfooter=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)

In [105]:
df['issue_d'].dt.year

0        2017
1        2017
2        2017
3        2017
4        2017
         ... 
96774    2017
96775    2017
96776    2017
96777    2017
96778    2017
Name: issue_d, Length: 96779, dtype: int64

In [0]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)

In [108]:
df['last_pymnt_d_month'] = df['last_pymnt_d'].dt.month
df['last_pymnt_d_month'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    4.0
Name: last_pymnt_d_month, dtype: float64

In [109]:
df['last_pymnt_d_year'] = df['last_pymnt_d'].dt.year
df['last_pymnt_d_year'].value_counts()

2017.0    79657
Name: last_pymnt_d_year, dtype: int64

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01

In [0]:
# int_rate = ' 13.56%'
# int_rate.strip('%')
# int_rate.strip().strip('%')
# float('8.5')